In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
os.chdir('C://Users//tjoseph//Downloads//SAC')


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(15,5)})

In [3]:
df=pd.read_csv('data.csv')
 

In [4]:

def find_best_method_for_group(group ):
    returns=[]
    ff=df[df['Group']==group]
    ff['eff']=ff['Returns']/ff['Actions']
    gf=ff.pivot_table(index='Day',columns='Method',values='eff'  )

    n_samples=90

    prior_A={'alpha':1,'beta':1}
    prior_B={'alpha':1,'beta':1}

    n_A=n_B=0
    return_A=return_B=0

    for i,j in zip(gf['A'],gf['B']):

        n_A=n_B=n_A+1
        
        

        return_A=return_A+i
        return_B=return_B+j



        posterior_A={}
        posterior_B={}
        posterior_A['alpha']=prior_A['alpha']+return_A
        posterior_A['beta']=prior_A['beta']+(n_A-return_A)

        posterior_B['alpha']=prior_B['alpha']+return_B
        posterior_B['beta']=prior_B['beta']+(n_B-return_B)


        c_A_samples=np.random.beta(a=posterior_A['alpha'],b=posterior_A['beta'],size=n_samples)
        c_B_samples=np.random.beta(a=posterior_B['alpha'],b=posterior_B['beta'],size=n_samples)
        

        p_AB=np.sum(c_A_samples>c_B_samples)/n_samples
        p_BA=np.sum(c_B_samples>c_A_samples)/n_samples




        #Expected loss
        loss_A=np.sum(np.maximum(0,(c_B_samples-c_A_samples)))/n_samples
        loss_B=np.sum(np.maximum(0,(c_A_samples-c_B_samples)))/n_samples

        t=0.001
        if loss_A<=t:
            print('# Days to run trials :',n_A-1)
            print('Switch to method A ')
            
            improvement=ff.loc[(ff['Day']>(n_A-1)) & (ff['Method']=='A')]['Returns'].sum()
            new_total_return=ff.loc[ff['Day']<=(n_A-1)]['Returns'].sum() + improvement*2
            print('Initial Total Returns : ', ff['Returns'].sum())
            print('New Total Returns : ', new_total_return)
            returns.append(new_total_return)


            break
        elif loss_B<=t:
            
            print('# Days to run trials :',n_A-1)
            print('Switch to method B')
            
            improvement=ff.loc[(ff['Day']>(n_A-1)) & (ff['Method']=='B')]['Returns'].sum()
            
            new_total_return=ff.loc[ff['Day']<=(n_A-1)]['Returns'].sum() + improvement*2
            
            print('Initial Total Returns : ', ff['Returns'].sum())
            print('New Total Returns : ', new_total_return)
            returns.append(new_total_return)
            
 
            break
    else:

        print('Continue test , need more data')
        returns.append(ff['Returns'].sum())
        
        
    return returns

              




In [23]:
kk={}
for group in range(0,1000):
    
    print('Group : ',group)
     
    kk[group]=find_best_method_for_group(group)[0]
    
    
    

Group :  0
# Days to run trials : 15
Switch to method A 
Initial Total Returns :  337
New Total Returns :  534
Group :  1
# Days to run trials : 15
Switch to method A 
Initial Total Returns :  492
New Total Returns :  839
Group :  2
# Days to run trials : 28
Switch to method A 
Initial Total Returns :  47
New Total Returns :  78
Group :  3
Continue test , need more data
Group :  4
# Days to run trials : 22
Switch to method B
Initial Total Returns :  910
New Total Returns :  1109
Group :  5
# Days to run trials : 17
Switch to method A 
Initial Total Returns :  866
New Total Returns :  1073
Group :  6
# Days to run trials : 12
Switch to method B
Initial Total Returns :  1125
New Total Returns :  1609
Group :  7
# Days to run trials : 79
Switch to method B
Initial Total Returns :  1043
New Total Returns :  1028
Group :  8
# Days to run trials : 29
Switch to method A 
Initial Total Returns :  82
New Total Returns :  110
Group :  9
# Days to run trials : 2
Switch to method B
Initial Total R

In [43]:
new_returns=sum(kk.values())
new_returns

631861

In [44]:
inital_returns=df['Returns'].sum()
inital_returns


526749

In [45]:
uplift= round(((new_returns-inital_returns)/inital_returns)*100,2)

In [46]:
print('Uplift(in %) = ',uplift )

Uplift(in %) =  19.95


#### To see results for random Groups inividually, use the below function:

In [53]:
group=np.random.randint(0,1000)
print('Group :' , group)
find_best_method_for_group(group)

Group : 166
# Days to run trials : 58
Switch to method B
Initial Total Returns :  244
New Total Returns :  262


[262]